# Data Science in Economics and Business - Class Project
In this class project, you are supposed to work with GDP data taken from the International Monetary Fund.

**IMPORTANT:** <br>
Please enter the matriculation number of all group members here:
1. XXXXXX
2. YYYYYY
3. ZZZZZZ


In this class project, you will use the different techniques taught in the course: data handling, data visualization, and machine learning.

First load the necessary packages. <br>
If you want to use additional libraries you can add them to the following cell:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import world_bank_data as wb
import wbgapi as wba
import datetime
sns.set()
import folium
from folium.plugins import MarkerCluster
import plotly.express as px
import logging
import country_converter as coco

pd.set_option('display.max_rows', None)
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

# Problem 1 - Data Handling
The basis of your work will be the following GDP dataset:

In [ ]:
gdp = pd.read_csv("GDP_IMF.csv", sep=";")
gdp.sample(20)

#### a)
As you can see, there are many missing values in the data. <br>
Before you can continue, you need to handle them. Proceed as follows:
- Delete those countries with no data at all (if any)
- Either delete the missing years for each country or use the years close by to approximate the missing value <br> *Example: Year 2004 is missing, but 2003 and 2005 are available; use the mean of GDP in 2003 and 2005 to replace missing year 2004.*

In [ ]:
gdp = gdp.drop(gdp[gdp['year']>2022].index)

In [ ]:
len(gdp['year'].unique())

In [ ]:
no_data_counts_country = gdp.groupby('Country')['GDP'].apply(lambda x: (x=='no data').sum())
no_data_counts_country[no_data_counts_country > 0]

In [ ]:
gdp = gdp.drop(gdp[gdp['Country'].isin(no_data_counts_country[no_data_counts_country >= 8].index)].index)

In [ ]:
len(gdp['Country'].unique())

In [ ]:
no_data_counts_year = gdp.groupby('year')['GDP'].apply(lambda x: (x=='no data').sum())
no_data_counts_year[no_data_counts_year > 0]

In [ ]:
# Replace 'no data' with nan values and convert to float
gdp['GDP'] = gdp['GDP'].replace('no data', np.nan).astype(float)

# Set multi-index and sort
gdp.set_index(['Country', 'year'], inplace=True)
gdp.sort_index(inplace=True)

# Generate a full index of all year combinations for each country
all_years = range(gdp.index.get_level_values('year').min(), gdp.index.get_level_values('year').max() + 1)
full_index = pd.MultiIndex.from_product([gdp.index.get_level_values('Country').unique(), all_years], 
                                        names=['Country', 'year'])

# Reindex and interpolate
gdp = gdp.reindex(full_index)
gdp['GDP'] = gdp.groupby('Country', group_keys=False)['GDP'].apply(lambda group: group.interpolate(method='linear', limit_direction='both'))

# Reset the index
gdp.reset_index(inplace=True)


#### b)
We are only interested in the data on a country level and for years already passed. <br>
The dataset, however, also contains information on whole regions such as the EU or G7. <br>
It also contains predictions for GDP in the upcoming years. <br>
Delete these observations from the data. <br>
*Hint: Get a list of all unique values of the Country column.*

In [ ]:
gdp['Country'].unique()

In [ ]:
country_mapping = {
    'Congo, Dem. Rep. of the': 'Democratic Republic of Congo',
    'Congo, Republic of ': 'Republic of Congo',
    'Taiwan Province of China': 'Taiwan SAR',
    'Lao P.D.R.': 'Lao People\'s Democratic Republic'
}

gdp['Country'] = gdp['Country'].replace(country_mapping)


In [ ]:
gdp['Country'] = gdp['Country'].str.split(',').str[0]

In [ ]:
gdp['Country'].unique()

In [ ]:
import requests
from bs4 import BeautifulSoup

# Send a GET request
response = requests.get('https://www.imf.org/en/Countries')

# Parse the response content with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the country names
countries = [country.text for country in soup.select('a[href^="/en/Countries/"]')]

In [ ]:
countries = countries[11:]

In [ ]:
countries = [c.replace('Congo, Democratic Republic of the', 'Democratic Republic of Congo') for c in countries]
countries = [c.replace('Congo, Republic of', 'Republic of Congo') for c in countries]
countries = [c.replace('Syrian Arab Republic', 'Syria') for c in countries]
countries = [c.replace('Hong Kong Special Administrative Region', 'Hong Kong SAR') for c in countries]
countries = [c.replace('Macao Special Administrative Region', 'Macao SAR') for c in countries]

countries = [c.replace('North Macedonia', 'North Macedonia ') for c in countries]
countries = [c.replace('Türkiye', 'Tuerkiye') for c in countries]
countries = [c.replace('Côte d\'Ivoire', 'Cote d\'Ivoire') for c in countries]
countries.append('Taiwan SAR')
countries.append('Puerto Rico')
countries.append('Saint Kitts and Nevis')
countries.append('Saint Lucia')
countries.append('Saint Vincent and the Grenadines')
countries.append('Sao Tome und Principe')
countries.append('West Bank and Gaza')


In [ ]:
countries = [country.split(',')[0] for country in countries]
len(countries)

In [ ]:
len(gdp['Country'].unique())

In [ ]:
missing_countries = gdp[~gdp['Country'].isin(countries)]

for country in missing_countries['Country'].unique():
    print(country)

In [ ]:
gdp = gdp[gdp['Country'].isin(countries)]

In [ ]:
len(gdp['Country'].unique())

In [ ]:
gdp['year'].unique()

In [ ]:
gdp.head()

# Problem 2 - Data Visualization
#### a)
To get some first insights in the data, create meaningful plots. <br>
You can use any kind that you deem useful: histograms, line plots, etc.

In [ ]:
# Use plotly express to create an interactive scatter plot
fig = px.scatter(gdp, x=gdp['year'], y=gdp['GDP'], color='Country', hover_name='Country')

fig.update_layout(
    xaxis_title=f'Year',
    yaxis_title=f'GDP',
    title='Countries GDP in Billions [USD] (1992-2023)',
    showlegend=True
)

# Show the interactive plot
fig.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(40, 8))
sns.boxplot(x='Country', y='GDP', data=gdp)
plt.title('Distribution of GDP Values for Each Country')
plt.xticks(rotation=90) # This makes the country names vertical so they don't overlap
plt.show()

In [ ]:
import matplotlib.colors as clrs


def heatmap_normal_scaling_ordered_alphabetical():
    pivot_data = gdp.pivot(index='Country', columns='year', values='GDP')

    plt.figure(figsize=(20, 40))  # Adjust the figure size as needed
    sns.heatmap(pivot_data, cmap='YlGnBu', cbar_kws={"shrink": 0.3})
    plt.xlabel('Year')
    plt.ylabel('Country')
    plt.title('GDP Heatmap by Country and Year')

    plt.show()

def heatmap_log_scaling_ordered_alphabetical():
    pivot_data = gdp.pivot(index='Country', columns='year', values='GDP').fillna(0)

    # Create a heatmap with log scaling for the legend
    plt.figure(figsize=(20, 40))
    sns.heatmap(pivot_data, cmap='YlGnBu', norm=clrs.LogNorm(vmin=1, vmax=pivot_data.values.max()), cbar_kws={"shrink": 0.3})
    plt.xlabel('Year')
    plt.ylabel('Country')
    plt.title('GDP Heatmap by Country and Year (Log Scaling)')


def heatmap_log_scaling_ordered_gdp(): 
    pivot_data = gdp.pivot(index='Country', columns='year', values='GDP').fillna(0)

    # Calculate the total GDP for each country and sort by GDP in descending order
    total_gdp = pivot_data.sum(axis=1)
    sorted_countries = total_gdp.sort_values(ascending=False).index

    # Reorder the rows in the pivot_data DataFrame based on the sorted countries
    pivot_data_sorted = pivot_data.loc[sorted_countries]

    # Create a heatmap with log scaling for the legend
    plt.figure(figsize=(20, 40))
    sns.heatmap(pivot_data_sorted, cmap='YlGnBu', norm=clrs.LogNorm(vmin=1, vmax=pivot_data.values.max()), cbar_kws={"shrink": 0.3})
    plt.xlabel('Year')
    plt.ylabel('Country')
    plt.title('GDP Heatmap by Country and Year in Billions [USD] (Sorted by total GDP over time Descending)')
    plt.show()


#heatmap_normal_scaling_ordered_alphabetical()
#heatmap_log_scaling_ordered_alphabetical()
heatmap_log_scaling_ordered_gdp()

In [ ]:

## Create a heatmap of the change in GDP from one year to the next for each country
dfh = gdp
# create a new column for the change in GDP from one year to the next
dfh['gdp_change'] = dfh.groupby('Country')['GDP'].diff()
dfh['gdp_change_category'] = np.where(dfh['gdp_change'] > 0, 'Increase', np.where(dfh['gdp_change'] < 0, 'Decrease', 'No Change'))

# create a mapper to map the gdp_change_category column to numerical values
mapper = {'Increase': 1, 'No Change': 0, 'Decrease': -1}

# Map the gdp_change_category column to numerical values
dfh['gdp_change_category'] = dfh['gdp_change_category'].map(mapper)

# Pivot the DataFrame to have years as columns and countries as index
df_pivot = dfh.pivot(index='Country', columns='year', values='gdp_change_category')

# Create a colormap
from matplotlib.colors import ListedColormap
cmap = ListedColormap(['red', 'white', 'green'])


## Calculate the absolute change in gdp from one year to the next as annotation for the heatmap
# Pivot the data to have each country's GDP as a separate column
pivot_data_labels = gdp.pivot(index='Country', columns='year', values='GDP')
# Calculate the change in GDP for each country from one year to the next
gdp_change_label = pivot_data_labels.diff(axis=1)


# Plot the heatmap
plt.figure(figsize=(35, 40))
sns.heatmap(df_pivot, annot=gdp_change_label, annot_kws={"fontsize":9}, cmap=cmap, cbar=False, linewidths=0.5, linecolor='lightgrey', fmt=".2f")
plt.xlabel('Year')
plt.ylabel('Country')
plt.title('GDP Change Heatmap in Billions [USD] (Green: Increase, Red: Decrease, White: No Change)')

plt.show()

In [ ]:
%%capture
import bar_chart_race as bcr

# Assuming you have already loaded the data into a DataFrame named 'data'
# Let's say the columns are: 'Country', 'Year', 'GDP'

# Pivot the data to have each country's GDP as a separate column
pivot_data_race = gdp.pivot(index='Country', columns='year', values='GDP')
pivot_data_race.columns = pd.to_datetime(pivot_data_race.columns, format='%Y')

# Transpose the DataFrame to have years as columns and countries as rows
pivot_data_race = pivot_data_race.T


# Create the bar chart race animation
bcr.bar_chart_race(
    df=pivot_data_race,
    filename='highest_gdp_comparison.gif',
    orientation='h',       # Horizontal bars
    sort='desc',           # Sort bars in descending order at each frame
    n_bars=20,             # Number of bars (countries) to include in each frame
    steps_per_period=10,   # Number of steps (frames) per year
    period_length=500,     # Length of each period (milliseconds)
    title='Top 20 Countries with Highest GDPs in Billions (USD) with median',
    bar_label_size=7,      # Font size of bar labels
    tick_label_size=7,     # Font size of tick labels
    period_fmt='%Y',       # Show only the year for periodic steps
    shared_fontdict={'family': 'Helvetica', 'color': '.1'},  # Font settings
    filter_column_colors=True,  # Apply the color from the final frame to all frames
    perpendicular_bar_func='median',  # Set the median as the reference line
)

![SegmentLocal](highest_gdp_comparison.gif "segment")

#### b)
Pick a year with as little missing values as possible. <br>
For this year, create an interactive map with `folium` that tells you the GDP in the country in the given year. <br>
*Hint: Be cautious with country names.*

In [ ]:
gdp_05 = gdp[gdp['year']==2005]

In [ ]:
gdp_05.shape

In [ ]:
from geopy.geocoders import Nominatim

# Initialize the geolocator
geolocator = Nominatim(user_agent="geoapiExercises")

# Create a world map
world_map= folium.Map(location=[10, -20], zoom_start=2.3)

# Loop over each country and its GDP
for idx, row in gdp_05.iterrows():
    # Get the location of the country
    location = geolocator.geocode(row['Country'])

    # If the location is found
    if location is not None:
        # Add a marker to the map
        folium.Marker(
            location=[location.latitude, location.longitude], 
            popup=f"Country: {row['Country']}, GDP: {row['GDP']}",
        ).add_to(world_map)

# Show the map
world_map


# Problem 3 - Supervised Machine Learning
#### a)
First, try to predict the GDP in a given year and in a given country using the data from the past years. <br>
Report the performance (measured in mean squared error) for different numbers of lags. 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
gdp = gdp.drop(['gdp_change', 'gdp_change_category'], axis=1)

In [ ]:
def train_model(df, country, model, lags):
    df_country = df[df['Country']==country].drop('Country', axis=1)
    df_country = df_country.sort_values(by='year').drop('year', axis=1)
    for lag in range(1, lags+1):
        df_country[f'GDP_lag_{lag}'] = df_country['GDP'].shift(lag)
    
    df_country = df_country.dropna()
    X_train, X_test, y_train, y_test = train_test_split(df_country.drop('GDP', axis=1),df_country['GDP'], test_size=.2,  shuffle=False)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    return mse

In [ ]:
linear = LinearRegression()
boost = AdaBoostRegressor(n_estimators=100, random_state=123)
forest = RandomForestRegressor(n_estimators = 50, random_state=123)
models = [linear, boost, forest]

In [ ]:
results = []
for country in gdp['Country'].unique():
    for lags in range (3, 6):
        row = {'Country': country, 'lag': lags}
        for model in models:
            mse = train_model(gdp, country, model, lags)
            results.append({'Country': country, 'lag': lags, 'model': type(model).__name__, 'mse': mse})

result_df = pd.DataFrame(results)

result_df.head()

In [ ]:
best_models_per_country = result_df.loc[result_df.groupby('Country')['mse'].idxmin()]
best_models_per_country

#### b)
Now, see if you can improve the prediction with additional data. <br>
In this task, you are supposed to be creative and use your intuition. What could be important predictors? Think of, for example:
- Country characteristics such as population, unemployment rates etc. <br> *Hint: You can search for official statistics, e.g., from OECD.*
- Major historical events such as wars, natural disasters etc. <br> *Hint: You can make dummy variables if such an event happened for given country and year.*
- Geographical information such as continent.

*Note: It is well possible that you can't find data on very small countries. If you don't find data for some countries, you can drop them.*

In [ ]:
current_year = datetime.datetime.now().year

In [ ]:
#Little test with wbgapi, to build directly a dataframe
# https://nbviewer.org/github/tgherzog/wbgapi/blob/master/examples/wbgapi-cookbook.ipynb
#wba.data.DataFrame(['SP.POP.TOTL', 'NY.GDP.PCAP.CD'],
#                  time=range(2010,2015), skipBlanks=True, columns='series')

In [ ]:
#Get countries from World Bank API and drop unnecessary data
countries = wb.get_countries()
countries = countries[countries['incomeLevel'] != 'Aggregates'].rename(columns={'name': 'countryName'})
countries.drop(['iso2Code', 'adminregion', 'capitalCity', 'lendingType', 'longitude', 'latitude'], axis=1, inplace=True)
countries.reset_index(level=0, inplace=True)
countries.head()

In [ ]:
data_series_dict = {
    #'Battle-related deaths (number of people)' : 'VC.BTL.DETH',
    #'Droughts, floods, extreme temperatures' : 'EN.CLC.MDAT.ZS',
    'Populalation (total)':'SP.POP.TOTL',
    'Labor force (total)' : 'SL.TLF.TOTL.IN',
    #'Real interest rate (%)':'FR.INR.RINR',                         #a lot worst performance
    'Inflation, consumer prices (annual %)':'FP.CPI.TOTL.ZG',
    'Imports of goods and services (in $)':'NE.IMP.GNFS.CD',
    'Export of goods and services (in $)':'NE.EXP.GNFS.CD'

	
}

df = pd.DataFrame()

for var_name, series_name in data_series_dict.items():
    #Get series with the dict
    data_series = wb.get_series(series_name, date='1992:'+str(current_year), id_or_value='id')

    # Reset index, drop 'Series' column, and rename the series column
    data_series_df = data_series.reset_index().drop(columns=['Series'])
    data_series_df.rename(columns={series_name: var_name}, inplace=True)

    # If result DataFrame is empty, copy the current data
    if df.empty:
        df = data_series_df.copy()
    else:
        # Otherwise merge current data with existing DataFrame
        df = pd.merge(df, data_series_df, on=['Country', 'Year'], how='outer')
df.head()

In [ ]:
# Merge country information into the df
df = pd.merge(df, countries, left_on='Country', right_on='id')
df.drop(['id'], axis=1, inplace=True)
#df.set_index(['Country', 'Year'], inplace=True)
df.head()

In [ ]:
list(df)

In [ ]:
#df['Battle-related deaths (number of people)'] = df['Battle-related deaths (number of people)'].notna().astype(int)

In [ ]:
#df['Droughts, floods, extreme temperatures'] = df['Droughts, floods, extreme temperatures'].notna().astype(int)

In [ ]:
#df['Real interest rate (%)'] = df.groupby('Country',group_keys=False)['Real interest rate (%)'].apply(lambda group: group.fillna(method='ffill'))
df['Inflation, consumer prices (annual %)'] = df.groupby('Country',group_keys=False)['Inflation, consumer prices (annual %)'].apply(lambda group: group.fillna(method='ffill'))
df['Labor force (total)'] = df.groupby('Country',group_keys=False)['Labor force (total)'].apply(lambda group: group.fillna(method='ffill'))

In [ ]:
missing_values_per_country = df.isnull().sum(axis=1).groupby(df['countryName']).sum()
missing_values_per_country.sort_values(ascending=False, inplace=True)
missing_values_per_country

In [ ]:
#df['Real interest rate (%)'] = df.groupby('Country',group_keys=False)['Real interest rate (%)'].apply(lambda group: group.fillna(method='ffill').fillna(method='bfill'))
df['Inflation, consumer prices (annual %)'] = df.groupby('Country',group_keys=False)['Inflation, consumer prices (annual %)'].apply(lambda group: group.fillna(method='ffill').fillna(method='bfill'))
df['Labor force (total)'] = df.groupby('Country',group_keys=False)['Labor force (total)'].apply(lambda group: group.fillna(method='ffill').fillna(method='bfill'))

In [ ]:
df = df[~df['Country'].isin(missing_values_per_country[missing_values_per_country > 8].index)]
df.shape

In [ ]:
missing_values = df.isnull().sum()
missing_values.sort_values(ascending=False, inplace=True)
missing_values

In [ ]:
#df = df.drop(missing_values[missing_values>500].index, axis=1)
#df = df.groupby('Country').filter(lambda x: ~x.isnull().all().any())
df.head()

In [ ]:
df.shape

In [ ]:
for column in data_series_dict.keys():
    print(column)
    if column in df.columns:
        print(column + ' inter')  # We don't want to interpolate the 'Country' column
        df[column] = df.groupby('Country', group_keys=False)[column].apply(lambda group: group.interpolate(limit_direction='both'))

In [ ]:
#Drop any country that has a fully empty column, as we cant interpolate then.
df = df.groupby('Country').filter(lambda x: ~x.isnull().all().any())

In [ ]:
missing_values = df.isnull().sum()
missing_values.sort_values(ascending=False, inplace=True)
missing_values

In [ ]:
# Filter rows containing at least one missing value
missing_values_sample = df[df.isnull().any(axis=1)]

# Display a sample of rows with missing values
missing_values_sample


In [ ]:
#Convert Countries to ISO3 to merge on later
gdp['Country'] = gdp['Country'].replace('Tuerkiye', 'Turkey')
converter = coco.CountryConverter()
gdp['countryKey'] = gdp['Country'].apply(lambda x: converter.convert(names=x, to='ISO3'))
gdp['Country'] = gdp['Country'].replace('Turkey', 'Tuerkiye')
gdp.sample(10)

In [ ]:
#Check for Countries that where unable to match
not_found = gdp[gdp['countryKey'] == 'not found']
not_found

In [ ]:
len(gdp['Country'].unique())

In [ ]:
#Merge the Countrys with the additional data
df.rename(columns={'Country': 'countryKey'}, inplace=True)
df['Year'] = df['Year'].astype(int)
gdp['year'] = gdp['year'].astype(int)
merged = pd.merge(df, gdp, how='inner', left_on=['countryKey', 'Year'], right_on=['countryKey', 'year'])
merged.head(20)

In [ ]:
len(merged['Country'].unique())

In [ ]:
#Encode the lables as int
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
merged['region'] = le.fit_transform(merged.region.values)
merged['incomeLevel'] = le.fit_transform(merged.incomeLevel.values)
merged.sample(5)


In [ ]:
#merged.drop(columns=['countryKey', 'countryName', 'Year',  'region', 'incomeLevel'], axis=1, inplace=True)
merged.drop(columns=['countryKey', 'countryName', 'Year'], axis=1, inplace=True)
merged.rename(columns={'Year': 'year'}, inplace=True)

In [ ]:
merged.head()

In [ ]:
merged.isnull().sum(axis=1).sum()

In [ ]:
results = []
for country in merged['Country'].unique():
    for lags in range (3, 6):
        row = {'Country': country, 'lag': lags}
        for model in models:
            mse = train_model(merged, country, model, lags)
            results.append({'Country': country, 'lag': lags, 'model': type(model).__name__, 'mse': mse})

result_df_new = pd.DataFrame(results)

result_df_new.head()

In [ ]:
best_models_per_country_new = result_df_new.loc[result_df_new.groupby('Country')['mse'].idxmin()]
best_models_per_country_new

In [ ]:
best_models_per_country[best_models_per_country['Country']=='China']

In [ ]:
best_models_per_country_new[best_models_per_country_new['Country']=='China']

In [ ]:
# Function to compare the mse from each dataframe
def compare_mse(row):
    if row['mse_old'] < row['mse_new']:
        return 0
    elif row['mse_new'] < row['mse_old']:
        return 1
    else:
        return -1
combined_df = best_models_per_country.merge(best_models_per_country_new, on='Country', suffixes=('_old', '_new'))

# Apply comparison function and create new column
combined_df['smaller_mse'] = combined_df.apply(compare_mse, axis=1)

# Create resulting dataframe
comparison_result = combined_df[['Country', 'smaller_mse', 'mse_old', 'mse_new']]


In [ ]:
comparison_result

In [ ]:
comparison_result['smaller_mse'].sum()

# Problem 4 - Unsupervised Machine Learning 
Use GDP and the information from Problem 3b) to cluster countries. <br>
Which is the optimal number of clusters? <br>
Can you provide an intuition for the clusters you identified?

In [ ]:
merged.head()

**Experiment 1**

In [ ]:
import pandas as pd

# Set the multi-index with 'Country' and 'year'
df_pivoted = merged.set_index(['Country', 'year'])

# Swap 'year' as a key level above the current columns
df_pivoted = df_pivoted.unstack(level='year')

# Swap the column names so that 'year' is on top
df_pivoted.columns = df_pivoted.columns.swaplevel(0, 1)

# Sort the columns in ascending order based on the 'year'
df_pivoted = df_pivoted.sort_index(axis=1)

# Display the DataFrame with the updated structure
df_pivoted.head()

In [ ]:
%%capture
# Standardize the data before PCA and clustering
#scaler = StandardScaler()
#df_scaled = pd.DataFrame(scaler.fit_transform(df_pivoted), columns=df_pivoted.columns)

# Determine the optimal number of clusters using the elbow method
inertia_values = []
max_clusters = 10
for k in range(1, max_clusters + 1):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_pivoted)
    inertia_values.append(kmeans.inertia_)

In [ ]:
# Plot the elbow graph to choose the optimal number of clusters (e.g., 3 or 4)
plt.figure(figsize=(8, 5))
plt.plot(range(1, max_clusters + 1), inertia_values, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia (Within-cluster Sum of Squares)')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.show()

In [ ]:
# Choose the optimal number of clusters based on the elbow graph
n_clusters = 3 # You can change this based on the elbow graph
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df_pivoted['Cluster'] = kmeans.fit_predict(df_scaled)

In [ ]:
df_pivoted.head(20)

In [ ]:
# Perform PCA
pca = PCA()
pca_result = pca.fit_transform(df_pivoted.values)
df_pivoted["PC1"] = pca_result[:,0]
df_pivoted["PC2"] = pca_result[:,1]
df_pivoted["PC3"] = pca_result[:,1]

# Plot the cumulative explained variance ratio
plt.figure(figsize=(8, 5))
explained_variance_ratio_cumsum = pca.explained_variance_ratio_.cumsum()
#plt.plot(range(1, len(explained_variance_ratio_cumsum)+1), explained_variance_ratio_cumsum, marker='o')
plt.plot(range(1, len(explained_variance_ratio_cumsum)+1), explained_variance_ratio_cumsum, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio for PCA')
plt.xlim(1, 10)
plt.show()

In [ ]:
df_pivoted.head(20)

In [ ]:
# Create an interactive scatter plot using plotly
fig = px.scatter_3d(df_pivoted, x='PC1', y='PC2',z='PC3', color='Cluster')
fig.update_layout(showlegend=True)

# Show the interactive plot
fig.show()